In [1]:
import sys
import os
from pathlib import Path

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

GNDTRUTH = Path('../data/raw/COMMON_images_masks/')
RAWIMGS = Path('../data/raw/GROUP_images/')
DATA = (RAWIMGS.parent).parent
MASKS = DATA/'masks'

import SimpleITK as sitk
from utils import *
from linear_registration import register_linear

In [2]:
fix_im = sitk.ReadImage(GNDTRUTH/'common_40_image.nii.gz')
mov_im = sitk.ReadImage(RAWIMGS/'g1_54_image.nii.gz')
fix_msk = sitk.ReadImage(MASKS/'common_40_regmask.nii.gz')
mov_msk = sitk.ReadImage(MASKS/'g1_54_mask.nii.gz')

# Arrays from images
fix_im_data = sitk.GetArrayFromImage(fix_im)
fix_msk_data = sitk.GetArrayFromImage(fix_msk)
mov_im_data = sitk.GetArrayFromImage(mov_im)
mov_msk_data = sitk.GetArrayFromImage(mov_msk)


In [3]:
registration = register_linear(fix_im, mov_im, fix_mask=fix_msk, mov_mask=mov_msk, verbose=True)

Cropping fixed image to ROI...
Cropping moving image to ROI...
Estimating affine transformation...
itk::simple::CompositeTransform
 CompositeTransform (0x2dce06e0)
   RTTI typeinfo:   itk::CompositeTransform<double, 3u>
   Reference Count: 1
   Modified Time: 118056
   Debug: Off
   Object Name: 
   Observers: 
     none
   TransformQueue: 
   >>>>>>>>>
   AffineTransform (0x2edd9f70)
     RTTI typeinfo:   itk::AffineTransform<double, 3u>
     Reference Count: 1
     Modified Time: 117902
     Debug: Off
     Object Name: 
     Observers: 
       none
     Matrix: 
       0.967118 0.032073 -0.0709902 
       0.00540326 1.00613 -0.0662785 
       0.0743118 0.132266 1.03884 
     Offset: [32.9518, 59.5222, -1596.65]
     Center: [-79.3516, -134.645, 836.698]
     Translation: [-28.1548, 2.81297, -1587.86]
     Inverse: 
       1.02903 -0.0416977 0.0676597 
       -0.010289 0.986057 0.0622077 
       -0.0723002 -0.122563 0.94985 
     Singular: 0
   TransformsToOptimizeFlags: 
           

In [6]:
mov_im_reg = registration['registered_image']
mov_msk_reg = registration['registered_mask']
lin_trfm = registration['affine_tfm']

In [7]:
mov_im_reg_data = sitk.GetArrayFromImage(mov_im_reg)
show_interactive_overlay(fix_im_data, mov_im_reg_data, 'coronal')

interactive(children=(IntSlider(value=256, description='slc', max=511), Output()), _dom_classes=('widget-inter…

<function utils.visualization.show_coronal_overlay(fix_arr, mov_arr, slc)>

In [8]:
sitk.WriteImage(mov_im_reg, DATA/'linear_registrations/with_regmask/g1_54_linreg_common40_img.nii.gz')
sitk.WriteImage(mov_msk_reg, DATA/'linear_registrations/with_regmask/g1_54_linreg_common40_msk.nii.gz')
sitk.WriteTransform(lin_trfm, DATA/'linear_registrations/with_regmask/transforms/g1_54_linreg_common40.tfm')

In [13]:
from utils.crop import downsample

mov_dspl = downsample(mov_im_reg, 4)
mov_msk_dspl = downsample(mov_msk_reg, 4)

In [14]:
mov_dspl_data = sitk.GetArrayFromImage(mov_dspl)
mov_msk_dspl_data = sitk.GetArrayFromImage(mov_msk_dspl)
show_interactive(mov_dspl_data, 'coronal', mov_msk_dspl_data)

interactive(children=(IntSlider(value=64, description='slc', max=127), Output()), _dom_classes=('widget-intera…

<function utils.visualization.show_coronal_slice(arr, slc, mask=None)>